In [1]:
def Maps(b, TI, TR, TE, param, slslice, boundaries):

    # Perform Non-Linear Least Squares Fit

    x0 = [300, 2, 1000, 100, 1, 1, 1,0,0,0]
    # for x[4], values of 0.01, 1 would lead to a straight line at 0, 
    # while 10 and 100 works fine

    lsr = least_squares(FitCost, x0, args=(b,TI, TR, TE, param, slslice), bounds=(boundaries), max_nfev=5000)
    cost = lsr.cost
    jac = lsr.jac
    values = lsr.x

    # Computing Mean Diffusivity
    # Diffusion Tensor
    D = [[values[4], values[7], values[8]], 
         [values[7], values[5], values[9]], 
         [values[8], values[9], values[6]]]

    # Eigenvectors and Eigenvalues 
    [evals , evecs] = la.eig(D)
    
    # Calculating MD 
    MD = (evals[0]+evals[1]+evals[2])/3
    FA = np.sqrt(1/2)*((np.sqrt((evals[0]-evals[1])**2+(evals[1]-evals[2])**2+(evals[2]-evals[0])**2))/np.sqrt(evals[0]**2+evals[1]**2+evals[2]**2))
    return MD, values[1],values[2],values[3], cost, D, FA

    # values[3]: T2* , [2]: T1, [1]: PD?, i.e. IE

In [2]:
# Fit Function

def Fit(x,b, TI, TR, TE, param):            

    S= np.abs(x[0] *(1-x[1]* np.exp(-TI/x[2])+np.exp(-TR/x[2]))* np.exp(-TE/x[3])* np.exp(-b*(np.square(param[:,0])*x[4]+np.square(param[:,1])*x[5]+np.square(param[:,2])*x[6]+2*param[:,0]*param[:,1]*x[7]+2*param[:,0]*param[:,2]*x[8]+2*param[:,1]*param[:,2]*x[9])))
    return S

# Simple Exponential Cost Function

def FitCost(x, b, TI, TR, TE, param, slslice):   
    # x[0] : Amplitude
    # x[1] : D value
    
    S= np.abs(x[0] *(1-x[1]* np.exp(-TI/x[2])+np.exp(-TR/x[2]))* np.exp(-TE/x[3])* np.exp(-b*(np.square(param[:,0])*x[4]+np.square(param[:,1])*x[5]+np.square(param[:,2])*x[6]+2*param[:,0]*param[:,1]*x[7]+2*param[:,0]*param[:,2]*x[8]+2*param[:,1]*param[:,2]*x[9])))
    res = slslice[0,:] -S
    return res